In [1]:
import pandas as pd
# import gensim

In [2]:
messages = pd.read_csv('SMSSpamCollection.txt', sep = '\t', names=['label','message'])

In [3]:
# DATA PREPREOCESSING
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    # review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = [lemmatizer.lemmatize(word) for word in review ] 
    review = ' '.join(review)
    corpus.append(review)

In [6]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

C:\Users\ADMIN\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [7]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [8]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [9]:
words = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [10]:
words[1]

['ok', 'lar', 'joking', 'wif', 'oni']

In [11]:
model = gensim.models.Word2Vec(words)

In [12]:
model.corpus_count

5569

In [13]:
model.epochs

5

In [14]:
# model.wv.similar_by_word('good')

In [15]:
# model.wv.similar_by_word('kid')

In [16]:
def avg_word2vec(doc):
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Trả về vector toàn số 0

In [17]:
from tqdm import tqdm

In [18]:
import numpy as np
X = []
for i in range(len(words)):
    X.append(avg_word2vec(words[i]))

In [19]:
len(X)

5569

In [20]:
# model.wv['good'].shape

In [21]:
# Independent Feature
X_new = np.array(X)

In [22]:
X_new.shape

(5569, 100)

In [23]:
# X_new[0]

In [24]:
X_new[0].shape

(100,)

In [25]:
# Dependent feature
y = messages[list(map(lambda x: len(x) > 0, corpus))]
y = pd.get_dummies(y['label']).astype(int)
y = y.iloc[:, 0].values

In [26]:
y.shape

(5569,)

In [27]:
# Reshape to converse 1 dimension to 2 demensions
# df = pd.DataFrame()
# for i in range(0, len(X)):
#     df = df.append(pd.DataFrame(X[i].reshape(1, -1)), ignore_index=True)

In [28]:
df_list = [pd.DataFrame(X[i].reshape(1, -1)) for i in range(len(X))]
df = pd.concat(df_list, ignore_index=True)

In [29]:
# df.shape

In [30]:
# df.head()

In [31]:
df['Output'] = y

In [32]:
# df.head()

In [33]:
# df.isnull().sum()

In [34]:
df.dropna(inplace = True)

In [35]:
# df.isnull().sum()

In [36]:
X = df.drop('Output', axis=1)

In [37]:
# X.isnull().sum()

In [38]:
y = df['Output']

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [40]:
# X_train

In [41]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [42]:
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [43]:
y_pred = classifier.predict(X_test)

In [44]:
y_pred

array([1, 1, 1, ..., 0, 1, 1])

In [45]:
from sklearn.metrics import accuracy_score, classification_report

In [46]:
accuracy_score(y_test, y_pred)

0.9676840215439856

In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88       161
           1       0.97      0.99      0.98       953

    accuracy                           0.97      1114
   macro avg       0.95      0.91      0.93      1114
weighted avg       0.97      0.97      0.97      1114



In [48]:
# Ví dụ email mới
new_email = "You have won a $1000 gift card. Click here to claim it now!"

# Bước 1: Xử lý dữ liệu email mới
def preprocess_email(email):
    # Loại bỏ ký tự không mong muốn
    email_cleaned = re.sub('[^a-zA-Z]', ' ', email)
    # Chuyển thành chữ thường và tách từ
    email_cleaned = email_cleaned.lower().split()
    # Lemmatization
    email_cleaned = [lemmatizer.lemmatize(word) for word in email_cleaned]
    return email_cleaned

# Bước 2: Chuyển đổi email thành vector Word2Vec
def get_avg_word2vec_for_email(email, model):
    email_words = preprocess_email(email)
    # Tạo embedding trung bình cho email
    email_vector = avg_word2vec(email_words)  # Sử dụng hàm avg_word2vec đã định nghĩa trước đó
    return email_vector

# Bước 3: Dự đoán bằng mô hình RandomForest
email_vector = get_avg_word2vec_for_email(new_email, model)
email_vector = email_vector.reshape(1, -1)  # Đảm bảo nó có đúng định dạng (1, số đặc trưng)

# Sử dụng mô hình để dự đoán
predicted_label = classifier.predict(email_vector)

# Kết quả dự đoán
if predicted_label[0] == 0:
    print("Đây là email spam.")
else:
    print("Đây là email bình thường (ham).")


Đây là email spam.


In [49]:
new_email = "'Hey mohan, can we get together to watch footbal game tomorrow?'"

email_vector = get_avg_word2vec_for_email(new_email, model)
email_vector = email_vector.reshape(1, -1)  # Đảm bảo nó có đúng định dạng (1, số đặc trưng)

# Sử dụng mô hình để dự đoán
predicted_label = classifier.predict(email_vector)

# Kết quả dự đoán
if predicted_label[0] == 0:
    print("Đây là email spam.")
else:
    print("Đây là email bình thường (ham).")

Đây là email bình thường (ham).
